<a href="https://colab.research.google.com/github/YuehMintTai/RPython/blob/main/Class08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#三軍總醫院北投分院統計及實驗設計課程之八
## 2021/7/13
## ytai1123@gmail.com
### 使用方法:


1.   使用gmail帳號登入
2.   按"執行階段" -->"全部執行" 以執行全部內容, 若要個別執行可點選每格程式左方箭頭或按 Control + Enter 鍵執行。

In [ ]:
##0-1
!git clone https://github.com/YuehMintTai/RPython.git

Cloning into 'RPython'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 53 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [ ]:
##0-2
!pip install rpy2

In [ ]:
##0-3
%load_ext rpy2.ipython

In [4]:
import tensorflow as tf
import torch
tf.__version__     ##2.5.0
torch.__version__   ##1.9.0+cu102

'1.9.0+cu102'

In [ ]:
##7-5 R^2 in Tensorflow
import tensorflow as tf
from tensorflow import feature_column
features=df['家庭滿意度apgar'] ## X
label=df['網路成癮分數YDQ']  ## Y
f_apgar=feature_column.numeric_column(key='家庭滿意度apgar')##,dtype=tf.float64,shape=())
feature_cols=[f_apgar]
label_ndarry=df['網路成癮分數YDQ']
def train_input():
  _dataset=tf.data.Dataset.from_tensor_slices(({'APGAR':df['家庭滿意度apgar']},label_ndarry))
  dataset=_dataset.batch(32)
  #ilterator=dataset.Dataset.iter()
  iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)
  features,labels=iterator.get_next()
  return features, labels
model5=tf.estimator.LinearRegressor(feature_columns=feature_cols)
model5.train(input_fn=train_input,steps=None)
##list(model5.predict())